<a href="https://colab.research.google.com/github/alcliberal/multirotor_sizing_isae_coa_2021/blob/main/05_SizingModelsIntroduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/alcliberal/multirotor_sizing_isae_coa_2021/blob/main/pictures/logo_sizinglab.png?raw=1" style="float:right; max-width: 60px; display: inline" alt="SizingLab" /></a>

# Sizing models for multi-rotor definition 
*Written by Marc Budinger (INSA Toulouse) and Scott Delbecq (ISAE-SUPAERO), Toulouse, France.*

## Global sizing procedure with XDSM representation

The following diagram represents the XDSM graph of the global sizing process of the multi-rotor. 

![XDSM](https://github.com/alcliberal/multirotor_sizing_isae_coa_2021/blob/main/pictures/xdsm_multirotor_base.png?raw=1)

> **Questions:**
* Give the main sizing problems you are able to detect.

It is the problem of algebric loop, which is, to calculate the mass we need the $M_{load}$ and then return to calculate the $M_{total} = M_{prop}+M_{motor}+M_{esc}+M_{bat}$.

* Propose one or multiple solutions (which can request equation manipulation, addition of design variables, addition of constraints) 

First of all, we can estimate $M_{load}=k_{load}M_{payload}$ where $1 \leq k_{load} \leq 10$. That implicated a constraint in the $M_{total}$ as it has to be lower to $M_{load}$.

### Solution using the Normalized Variable Hybrid formulation

The following diagram represents the XDSM graph of the global sizing process of the multi-rotor with the NVH formulation to achieve the MultiDisciplinary Analysis (MDA). 

![XDSM](https://github.com/alcliberal/multirotor_sizing_isae_coa_2021/blob/main/pictures/xdsm_multirotor_mda.png?raw=1)

It consists in addition an oversizing coefficent $k_{os} \quad [1-10]$ as a design variable and additional inequality constraint $M_{total_{load}} \geq M_{tot}$. This way, we make sure that the drone is capable of lifting the load and itself. As we are minimizing the total mass $M_{tot}$ the oversizing coefficient $k_{os}$ will tend to be as small as possible and thus the inequality constraint will be equivalent to an equality constraint ($M_{total_{load}} = M_{tot}$). This is one way of solving an algebraic loop, other methods use numerical solvers or other optimizer-based formulations.

The quadrotor sizing problem contains other solvability issues such as overconstrained singularities. Try to utilize the NVH formulation to solve them.

## Collaborative work

The goal now is to work in a collaborative manner to quickly implement the global sizing procedure.
You will work in teams and in each team will work on a sub-part:
* [Propeller selection](06a_PropellerSelection.ipynb)
* [Motor selection](06b_MotorSelection.ipynb)
* [Battery and ESC selection](06c_BatteryESCSelection.ipynb)
* [Frame definition](06d_FrameSelection.ipynb)

In each of the following notebooks you will complete the code of a sizing brick that will be defined by already selected inputs and outputs. Thank you for respecting them to ensure a successful final assembly of these bricks in the complete optimization problem.

#### Main problems to be solved

In general, the establishment of a sizing procedure involves the resolution of the following 3 problems:
* a set of equations sub-constrained by the addition of a design variable in the optimization problem;
* an over-constrained variable by adding a design variable (multiplier) and the transfer of the excess equation(s) in the constrained part of the optimization problem;
* an algebraic loop by the use of a simplified equation weighted by a multiplying coefficient and a constraint representing the initial equation.


#### Caution for the optimizer

Where possible:
* the design variables must take the form of a normalized variable around 1 (oversize coef. for example) or easily bounded to facilitate the work of the optimizer.
* the constraints must take the form of inequality and not of equality which often introduce numerical difficulties. The optimization of the objective (for example the total mass) will certainly force some (active) constraints to come to an end. 

### Example of generic sizing code

Below, you will find an example of an optimization code to solve the following problem.  

>For a tank of $x,y,z$ dimensions with $x\in[0.1,1]$, $y\in[0.1,1]$ and $z\in[0.1,2.5]$ m:   
Minimize the surface $xy+2z(x+y)$ 
and 
respect the constraints      
 - volume $xyz$ bigger than 200 l.  
 - developped dimensions $2z+x$ and $2z+y$ lower than 1.5m for machining feasability.  
   
First work is to define a `SizingCode` function which enables:
- to calculate the objective to minimize
- to calculate the constraints
- to print the results


In [8]:
!pip3 install ipywidgets

In [10]:
import pandas as pd

import ipywidgets as widgets
from ipywidgets import interactive

pd.options.display.float_format = '{:,.2f}'.format


# -----------------------
# sizing code
# -----------------------
# inputs: 
# - param: optimisation variables vector (reduction ratio, oversizing coefficient)
# - arg: selection of output  
# output: 
# - objective if arg='Obj', problem characteristics if arg='Prt', constraints other else

def SizingCode(param, arg):
# Design variables in param
# ---
    x = param[0] # variable x
    y = param[1] # variable y 
    z = param[2] # variable z
    
# Calculus 
# ---
    volume = x*y*z
    surface = x*y+2*(x+y)*z
        
# Objective and Constraints sum up
# ---
    objective = surface
    
    constraints = [volume
                   -.2,
                   1.5-(2*z+x),
                   1.5-(2*z+y) ]
    
# Returns selection
# -------------------
    if arg == 'Obj':
        return objective

    elif arg=='Prt':
    # the data to print a defined into a Pandas dataframe
        col_names = ['Type', 'Name', 'Value', 'Unit', 'Comment']

        df = pd.DataFrame()
    
      
        df = df.append([{'Type': 'Objective', 'Name': 'Surface', 'Value': objective, 'Unit': '[m^2]', 'Comment': 'Mini surface'}])[col_names]
        df = df.append([{'Type': 'Constraints', 'Name': 'Const 1', 'Value': constraints[0], 'Unit': '[m^3]', 'Comment': 'Volume'}])[col_names]
        df = df.append([{'Type': 'Constraints', 'Name': 'Const 2', 'Value': constraints[1], 'Unit': '[m]', 'Comment': 'Developped length'}])[col_names]
        df = df.append([{'Type': 'Constraints', 'Name': 'Const 3', 'Value': constraints[2], 'Unit': '[m]', 'Comment': 'Developped length'}])[col_names]
        df = df.append([{'Type': 'Variables', 'Name': 'x', 'Value': x, 'Unit': '[m]', 'Comment': 'x'}])[col_names]
        df = df.append([{'Type': 'Variables', 'Name': 'y', 'Value': y, 'Unit': '[m]', 'Comment': 'y'}])[col_names]
        df = df.append([{'Type': 'Variables', 'Name': 'z', 'Value': z, 'Unit': '[m]', 'Comment': 'z'}])[col_names]
        df = df.append([{'Type': 'Tank', 'Name': 'Volume', 'Value': volume, 'Unit': '[m^3]', 'Comment': 'Tank volume'}])[col_names]
        df = df.append([{'Type': 'Tank', 'Name': 'Surface', 'Value': surface, 'Unit': '[m^2]', 'Comment': 'Tank surface'}])[col_names]
        df = df.append([{'Type': 'Tank', 'Name': 'Developped length', 'Value': 2*z+x, 'Unit': '[m]', 'Comment': 'x length'}])[col_names]        
        df = df.append([{'Type': 'Tank', 'Name': 'Developped length', 'Value': 2*z+y, 'Unit': '[m]', 'Comment': 'y length'}])[col_names]

    # the dataframe is then organised to be printed with interactive widgets
        items = sorted(df['Type'].unique().tolist())

        def f(Type):
            return df[df['Type']==Type] 
        widgets.interact(f, Type=items)
        return f
        
    else:
        return constraints

ModuleNotFoundError: No module named 'ipywidgets'

We will now use the [optimization algorithms](https://docs.scipy.org/doc/scipy/reference/optimize.html) of the Scipy package to solve and optimize the configuration. We use here the SLSQP algorithm without explicit expression of the gradient (Jacobian). For global evaluation of the solution, you can use the differential evolution algorithm.


In [2]:
import scipy
import scipy.optimize

# Vector of initial parameters
parameters = scipy.array((1,1,1))

# Optimization with SLSQP algorithm: definition of requested functions
contrainte = lambda x: SizingCode(x, 'Const')
objectif = lambda x: SizingCode(x, 'Obj')
objectifP = lambda x: SizingCode(x, 'ObjP')

# Optimization bounds
bounds = [(.1,1), (.1,1), (0.1,2.5)]

# SLSQP omptimisation
result = scipy.optimize.fmin_slsqp(func=objectif, x0=parameters, 
                                   bounds=bounds,
                                   f_ieqcons=contrainte, iter=1500, acc=1e-12)


# Final characteristics after optimization 
print("-----------------------------------------------")
print("Final characteristics after optimization :")

print(SizingCode(result, 'Obj'))
SizingCode(result, 'Prt')


Optimization terminated successfully.    (Exit mode 0)
            Current function value: 1.6286505699551932
            Iterations: 13
            Function evaluations: 58
            Gradient evaluations: 11
-----------------------------------------------
Final characteristics after optimization :
1.6286505699551932


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead
  """


interactive(children=(Dropdown(description='Type', options=('Constraints', 'Objective', 'Tank', 'Variables'), …

<function __main__.SizingCode.<locals>.f>